In [6]:
!pip install overturemaps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 35.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2


## Download and get overture Maps Data

In [7]:
import overturemaps
table = overturemaps.record_batch_reader("administrative_boundary").read_all()

table = table.combine_chunks()

## Save overture data to parquet

In [8]:
import pyarrow.parquet as pq
pq.write_table(table, 'adb.parquet')

## Get Nepal Boundary

Getting boundary from OSM : https://www.openstreetmap.org/relation/184633

In [13]:
import requests
import json
response = requests.get("https://api-prod.raw-data.hotosm.org/v1/osm_id/?osm_id=184633")
with open("nepal.geojson", "w+") as f:
    f.write(json.dumps(response.json()))

## Create duck db instance and load downloaded overture map data

DuckDB can directly load arrow tables , For sake of this tutorial data is saved to parquet to showcase how can we load downloaded parquet directly

In [10]:
import duckdb

# Connect to the DuckDB database
con = duckdb.connect('adb.db')

con.execute(f'CREATE TABLE adb AS SELECT * FROM read_parquet("adb.parquet")')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## Install spatial extension

https://duckdb.org/docs/extensions/spatial.html

In [11]:
con.execute('install spatial;')
con.execute('load spatial;')

## Create table from our boundary nepal geojson
Duckdb supports several geospatial functions as well as input like one in the example below

In [14]:
con.execute('create table nepal as select * from st_read("nepal.geojson")')

## Describe your table to know column and their formats

In [ ]:
con.sql('describe adb;')

┌──────────────────────┬───────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name      │                      column_type                      │  null   │   key   │ default │  extra  │
│       varchar        │                        varchar                        │ varchar │ varchar │ varchar │ varchar │
├──────────────────────┼───────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ id                   │ VARCHAR                                               │ YES     │ NULL    │ NULL    │ NULL    │
│ geometry             │ BLOB                                                  │ YES     │ NULL    │ NULL    │ NULL    │
│ bbox                 │ STRUCT(xmin FLOAT, xmax FLOAT, ymin FLOAT, ymax FLO…  │ YES     │ NULL    │ NULL    │ NULL    │
│ admin_level          │ INTEGER                                               │ YES     │ NULL    │ NULL    │ NULL    │
│ is_maritime          │ BOOLEAN

## Lets write some spatial query
Lets filter only those feature which intersects with our boundary

In [ ]:
query = f"""select a.id,a.admin_level,a.version,a.population,a.default_language,ST_GeomFromWKB(a.geometry) as geom  from adb as a , nepal as b where ST_Intersects(ST_GeomFromWKB(a.geometry), ST_GeomFromWKB(ST_AsWKB(b.geom)))"""

In [ ]:
con.sql(query)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────┬─────────────┬─────────┬────────────┬──────────────────┬───────────────────────────────────────┐
│          id          │ admin_level │ version │ population │ default_language │                 geom                  │
│       varchar        │    int32    │  int32  │   int32    │     varchar      │               geometry                │
├──────────────────────┼─────────────┼─────────┼────────────┼──────────────────┼───────────────────────────────────────┤
│ 0854f716bfffffff01…  │           3 │       0 │       NULL │ NULL             │ LINESTRING (81.0483975 28.4092009, …  │
│ 08514986bfffffff01…  │           2 │       0 │       NULL │ NULL             │ LINESTRING (83.8557792 27.3518667, …  │
│ 0851280b7fffffff01…  │           3 │       0 │       NULL │ NULL             │ LINESTRING (82.4554416 27.1507079, …  │
│ 0854706d3fffffff01…  │           3 │       0 │       NULL │ NULL             │ LINESTRING (81.7427347 27.9536411, …  │
│ 085378597fffffff01…  │        

## Export them in GIS formats

DuckDB has support for several geospatial data formats with the help of OGR , Find more : https://duckdb.org/docs/extensions/spatial.html

In [ ]:
con.execute(f"COPY ({query.strip()}) to 'ad_nepal.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON', LAYER_CREATION_OPTIONS 'WRITE_BBOX=YES');")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))